In [1]:
%matplotlib inline
import torch
import os,sys
import numpy as np
import torchvision.io as io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torchvision import transforms 
from torch.utils.data import Dataset, random_split

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [83]:
if "google.colab" in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True, use_metadata_server=False)
    
def import_file(url, filename):
  import requests
  r = requests.get(url)

  with open(filename, 'w') as f:
        f.write(r.text)

if "google.colab" in sys.modules:
    # Save datagenerators as file to colab working directory
    # If you are using GitHub, make sure you get the "Raw" version of the code
    url_u_net = 'https://raw.githubusercontent.com/karimassi/road-segmentation/u_net/src/models/u_net.py?token=AJVSYDCYFSNCXQW2B5UOPG27ZJREK'
    import_file(url_u_net, 'u_net.py')

    url_u_net_paper = 'https://raw.githubusercontent.com/karimassi/road-segmentation/u_net/src/models/u_net_paper.py?token=AJVSYDBJJBS6FXDVBFXKV7S7ZJRGY'
    import_file(url_u_net_paper, 'u_net_paper.py')

    url_training = 'https://raw.githubusercontent.com/karimassi/road-segmentation/main/src/training.py?token=AFUMR6FMXJI4DQHDQGBSNEK7YYMNU'
    import_file(url_training, 'training.py')

    url_image_mask_dataset = "https://raw.githubusercontent.com/karimassi/road-segmentation/u_net/src/image_mask_dataset.py?token=AJVSYDDNI2HGBE2YKVRZXFK7ZWOBA"
    import_file(url_image_mask_dataset, 'image_mask_dataset.py')
        
    import u_net
    import u_net_paper 
    import training
    import image_mask_dataset
else:
    from src import u_net
    from src import u_net_paper
    from src import training
    from src import image_mask_dataset

Mounted at /content/drive


In [84]:
import image_mask_dataset

root_dir = "/content/drive/Shareddrives/road-segmentation/data/training/"
image_dir = root_dir + "images/"
gt_dir = root_dir + "groundtruth/"

num_epochs = 10
learning_rate = 1e-3
batch_size = 5

dataset = image_mask_dataset.ImageMaskDataset(image_dir, gt_dir)

data_len = len(dataset)
train_len = int(data_len * 0.2)
test_len = int(data_len * 0.8)

dataset_train, dataset_test = random_split(dataset, [train_len, test_len])
print(len(dataset_train), len(dataset_test))

dataloader_train = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=True
)

from u_net import UNet

NUM_CHANNELS = 3
NUM_FILTERS = 32

# TODO: implement training for u-net model
# Train the logistic regression model with the Adam optimizer
criterion = torch.nn.MSELoss()
model = UNet(NUM_CHANNELS, NUM_FILTERS).to(device)

#from torchsummary import summary
#summary(model, input_size=(NUM_CHANNELS, 400, 400))

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
training.train(model, criterion, dataloader_train, dataloader_test, optimizer, num_epochs)

20 80
Starting training
Epoch  1 | Test accuracy : 0.0 | Test F1 : 0.27441


KeyboardInterrupt: ignored